In [42]:
%pip install sentiment-analysis-spanish

   ---------------------------------------- 0.0/30.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.0 MB 660.6 kB/s eta 0:00:46
   ---------------------------------------- 0.2/30.0 MB 2.5 MB/s eta 0:00:12
   - -------------------------------------- 1.5/30.0 MB 10.5 MB/s eta 0:00:03
   ------ --------------------------------- 4.6/30.0 MB 24.4 MB/s eta 0:00:02
   ------- -------------------------------- 5.6/30.0 MB 23.8 MB/s eta 0:00:02
   ------------- -------------------------- 10.3/30.0 MB 46.7 MB/s eta 0:00:01
   ---------------- ----------------------- 12.2/30.0 MB 65.2 MB/s eta 0:00:01
   ----------------------- ---------------- 17.5/30.0 MB 72.6 MB/s eta 0:00:01
   ---------------------------- ----------- 21.1/30.0 MB 72.6 MB/s eta 0:00:01
   --------------------------------- ------ 24.8/30.0 MB 81.8 MB/s eta 0:00:01
   ------------------------------------- -- 28.3/30.0 MB 73.1 MB/s eta 0:00:01
   ---------------------------------------  30.0/30.0 MB 72.6 MB/s

In [26]:
import pandas as pd
from ntscraper import Nitter

scraper = Nitter(log_level= 0, skip_instance_check=False)

Testing instances: 100%|██████████| 16/16 [00:30<00:00,  1.93s/it]


In [27]:

def get_tweets(query: str, mode: str = "term", num_tweets: int = 100, lang = 'es'):
    '''
    Main function for web scraping tweets.

    Args:
        query: Words that are going to be searched
        mode: Searching query as a "term", "hashtag" or a "user" tweet
        num_tweets: Number of tweets to be scraped
        lang: Language of the tweets to be scraped
    '''
    tweets = scraper.get_tweets(terms= query, mode= mode, number= num_tweets, language= lang)
    return tweets

In [28]:
tweets = get_tweets(query="perro sanche")

09-Sep-24 09:31:24 - Empty page on https://nitter.privacydev.net


In [29]:
tweets

{'tweets': [{'link': 'https://twitter.com/mojonpalacaspa/status/1833032652296683684#m',
   'text': '¿Viene con PerroSanche? ¿Por qué no la Argentina de Milei?',
   'user': {'name': 'Tía Kika Perninas',
    'username': '@mojonpalacaspa',
    'profile_id': '1765296281407094784',
    'avatar': 'https://pbs.twimg.com/profile_images/1765296281407094784/MKTWGHwv_bigger.jpg'},
   'date': 'Sep 9, 2024 · 6:40 AM UTC',
   'is-retweet': False,
   'is-pinned': False,
   'external-link': '',
   'replying-to': ['@iescolar'],
   'quoted-post': {},
   'stats': {'comments': 0, 'retweets': 0, 'quotes': 0, 'likes': 0},
   'pictures': [],
   'videos': [],
   'gifs': []},
  {'link': 'https://twitter.com/7fcbarcelonista/status/1833018389545074847#m',
   'text': 'Me he casado, hipotecado y con 2 hijos… antes tenía tiempo y dinero… ahora ni uno ni lo otro… no todo x culpa de Perro Sánche pero algo ha ayudado xD',
   'user': {'name': 'Fcbarcelonista21',
    'username': '@7fcbarcelonista',
    'profile_id': '94

In [30]:
final_tweets = []
for tweet in tweets['tweets']:
    data = [tweet['user']['profile_id'], tweet['text']]
    final_tweets.append(data)

In [31]:
data = pd.DataFrame(final_tweets, columns = ['profile_id', 'text'])

In [50]:
import re
from nltk.corpus import stopwords
import nltk
import emoji

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

def give_emoji_free_text(self, text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
        
    return clean_text

def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # Deletes URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Deletes mentions
    tweet = re.sub(r'#\w+', '', tweet)  # Deletes hashtags
    tweet = re.sub(r'\d+', '', tweet)  # Deletes numbers
    tweet = tweet.lower()  # Converts text into lowcase
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
    tweet = emoji.replace_emoji(tweet, replace= '') # Deletes emojis
    return tweet

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
from textblob import TextBlob

blob = TextBlob
def get_sentiment(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

for tweet in data['text']:
    cleaned_tweet = clean_tweet(tweet)
    sentiment = get_sentiment(cleaned_tweet)
    ## Detect language -> Choose library to detect the sentiment
    print(f'\nTweet: {cleaned_tweet}\nSentiment: {sentiment}\n --------------------------------')


Tweet: ¿viene perrosanche? ¿por argentina milei?
Sentiment: 0.0
 --------------------------------

Tweet: casado, hipotecado hijos… tiempo dinero… ahora otro… x culpa perro sánche ayudado xd
Sentiment: 0.0
 --------------------------------

Tweet: perro sanche siempre gana. siempre.
Sentiment: 0.0
 --------------------------------

Tweet: joda killcam perro sanche supongo cine
Sentiment: 0.0
 --------------------------------

Tweet: mal aclimatados, culpa perro sanche director. 
Sentiment: 0.0
 --------------------------------

Tweet: votado puesto psoe gobierno, gañán
Sentiment: 0.0
 --------------------------------

Tweet: si, vendido "si votáis, viene ultraderecha fuertes frenarla", entonces toreado querido.
Sentiment: 0.0
 --------------------------------

Tweet: tontos de...." mientras gobierne derecha"....
Sentiment: 0.0
 --------------------------------

Tweet: realidad si votaste partido pensando em votar izquierda realidad votado psoe gobierna beneplácito resto. así sí, votad